### Домашнє завдання
В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням.

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статобою.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння.

Порівняння моделей на основі однієї метрики(такої як Accuracy)- не приймається. Дз повинно бути виконано у Jupyter Nootebook,(або Google Colab) і задеплоїне на Гітхаб у вигляді файлу .ipynb.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from pathlib import Path
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

Будуємо датасет з усіх файлів (може зайняти кілька хвилин):

In [2]:
path = Path("./data")
df = pd.DataFrame()
for el in path.iterdir():
    if el.is_dir():
        for elem in el.iterdir():
            if elem.is_file():
                tmp = pd.read_csv(elem)
                splited_name = elem.stem.rsplit("-")
                tmp.insert(0,"time", int(splited_name[1]))
                tmp.insert(0,"action", el.name)
                df = pd.concat([df, tmp], axis=0, ignore_index=True)
df = df.sort_values(by=["action", "time"], ignore_index=True)
df

,action,time,accelerometer_X,accelerometer_Y,accelerometer_Z
0,idle,1,1.000776,4.616021,8.576031
1,idle,1,0.718261,4.209007,8.446744
2,idle,1,-0.909797,-0.282516,9.203311
3,idle,1,5.099650,0.148441,8.418014
4,idle,1,1.762132,-0.162806,9.251195
...,...,...,...,...,...
193855,walking,1850,4.529830,-12.488156,-0.378284
193856,walking,1850,6.871359,-11.037270,4.620809
193857,walking,1850,7.259220,-6.244078,1.355118
193858,walking,1850,6.143522,0.550666,11.391612


Виконуємо обчислення та будуємо новий датасет з time domain features на основі даних попереднього. Фактично, це відомі (одні більше, інші менше) функції мінімуму, максимуму, медіани, мат. очікування, стандартного відхилення, коефіцієнту асиметрії, двійкової ентропії та міжквартильного розмаху вибірки, розраховані на множині 30-семплових наборів значень прискорення для кожної 3D-осі акселерометра окремо, що відповідають окремим файлам отриманих даних.

In [3]:
df_stats = df.groupby(["action", "time"]).agg(acc_X_min = ('accelerometer_X', 'min'),
                                   acc_X_max = ('accelerometer_X', 'max'),
                                   acc_X_median = ('accelerometer_X', 'median'),
                                   acc_X_mean = ('accelerometer_X', 'mean'),
                                   acc_X_std = ('accelerometer_X', 'std'),
                                   acc_X_skew = ('accelerometer_X', 'skew'),
                                   acc_X_entropy = ('accelerometer_X', lambda x : entropy(x.value_counts(), base=2)),
                                   acc_X_IQR = ('accelerometer_X', lambda x : x.quantile(0.75) - x.quantile(0.25)),
                                   acc_Y_min = ('accelerometer_Y', 'min'),
                                   acc_Y_max = ('accelerometer_Y', 'max'),
                                   acc_Y_median = ('accelerometer_Y', 'median'),
                                   acc_Y_mean = ('accelerometer_Y', 'mean'),
                                   acc_Y_std = ('accelerometer_Y', 'std'),
                                   acc_Y_skew = ('accelerometer_Y', 'skew'),
                                   acc_Y_entropy = ('accelerometer_Y', lambda x : entropy(x.value_counts(), base=2)),
                                   acc_Y_IQR = ('accelerometer_Y', lambda x : x.quantile(0.75) - x.quantile(0.25)),
                                   acc_Z_min = ('accelerometer_Z', 'min'),
                                   acc_Z_max = ('accelerometer_Z', 'max'),
                                   acc_Z_median = ('accelerometer_Z', 'median'),
                                   acc_Z_mean = ('accelerometer_Z', 'mean'),
                                   acc_Z_std = ('accelerometer_Z', 'std'),
                                   acc_Z_skew = ('accelerometer_Z', 'skew'),
                                   acc_Z_entropy = ('accelerometer_Z', lambda x : entropy(x.value_counts(), base=2)),
                                   acc_Z_IQR = ('accelerometer_Z', lambda x : x.quantile(0.75) - x.quantile(0.25)),
                                  ).reset_index()

Розбиваємо вибірку на окремі вибірки з ознаками та цільовою змінною відповідно:

In [4]:
y = df_stats["action"]
X = df_stats.drop(columns=["action"])

Виділяємо окремо навчальні вибірки та тестові:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
base_features = [
    "acc_X_min",
    "acc_X_max",
    "acc_X_median",
    "acc_X_mean",
    "acc_Y_min",
    "acc_Y_max",
    "acc_Y_median",
    "acc_Y_mean",
    "acc_Z_min",
    "acc_Z_max",
    "acc_Z_median",
    "acc_Z_mean",
]
additional_features = {
    "no additional features": [],
    "std": ["acc_X_std", "acc_Y_std", "acc_Z_std"],
    "skew": ["acc_X_skew", "acc_Y_skew", "acc_Z_skew"],
    "entropy": ["acc_X_entropy", "acc_Y_entropy", "acc_Z_entropy"],
    "IQR": ["acc_X_IQR", "acc_Y_IQR", "acc_Z_IQR"],
    "all features": [
        "acc_X_std",
        "acc_Y_std",
        "acc_Z_std",
        "acc_X_skew",
        "acc_Y_skew",
        "acc_Z_skew",
        "acc_X_entropy",
        "acc_Y_entropy",
        "acc_Z_entropy",
        "acc_X_IQR",
        "acc_Y_IQR",
        "acc_Z_IQR",
    ],
}

Будуємо SVM (Support Vector Machine) класифікатор та сітку параметрів для навчання:

In [7]:
svm_param_dist = {
    "estimator__n_epochs": list(np.arange(10, 50, 10)),
    "estimator__lr_all": ["linear", "poly", "rbf", "sigmoid"],
    "estimator__reg_all": ["linear", "poly", "rbf", "sigmoid"],
}
svm_classifier = OneVsRestClassifier(
    SVC(probability=True, tol=0.0001, random_state=42), n_jobs=8
)
svm_grid_search = GridSearchCV(
    svm_classifier,
    param_grid=svm_param_dist,
    scoring=["accuracy", "f1_macro"],
    refit="f1_macro",
    cv=3,
)

Проводимо навчання SVM класифікатора з різними параметрами та з використанням різних наборів ознак (може зайняти кілька десятків хвилин):

In [8]:
svm_classifiers = {}
for key in additional_features.keys():
    svm_grid_search.fit(X_train[base_features + additional_features[key]], y_train)
    y_pred = svm_grid_search.predict(X_test[base_features + additional_features[key]])
    print(f"SVM classifier with {key}:")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.5f}")
    f1_macro = f1_score(y_test, y_pred, average="macro")
    print(f"f1_macro: {f1_macro:.5f}")
    report = classification_report(y_test, y_pred)
    print(report)
    svm_classifiers[key] = svm_grid_search

SVM classifier with no additional features:
Accuracy: 0.98840
f1_macro: 0.93142
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       0.85      0.67      0.75        33
     walking       0.97      0.99      0.98       370

    accuracy                           0.99      1293
   macro avg       0.95      0.91      0.93      1293
weighted avg       0.99      0.99      0.99      1293

SVM classifier with std:
Accuracy: 0.98840
f1_macro: 0.93142
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       0.85      0.67      0.75        33
     walking       0.97      0.99      0.98       370

    accuracy                           0.99      1293
   macro avg       0.95      0.91      0.93      1293
weighted avg       0.99      0.99      0.99 

Будуємо RF (Random Forest) класифікатор та сітку параметрів для навчання:

In [9]:
rf_param_dist = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

rf_classifier = RandomForestClassifier(random_state=42, n_jobs=8)

rf_grid_search = GridSearchCV(
    rf_classifier,
    param_grid=rf_param_dist,
    scoring=["accuracy", "f1_macro"],
    refit="f1_macro",
    cv=3,
)

Проводимо навчання RF класифікатора з різними параметрами та з використанням різних наборів ознак (може зайняти кілька десятків хвилин):

In [10]:
rf_classifiers = {}
for key in additional_features.keys():
    rf_grid_search.fit(X_train[base_features + additional_features[key]], y_train)
    y_pred = rf_grid_search.predict(X_test[base_features + additional_features[key]])
    print(f"RF classifier with {key}:")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.5f}")
    f1_macro = f1_score(y_test, y_pred, average="macro")
    print(f"f1_macro: {f1_macro:.5f}")
    report = classification_report(y_test, y_pred)
    print(report)
    rf_classifiers[key] = rf_grid_search

RF classifier with no additional features:
Accuracy: 1.00000
f1_macro: 1.00000
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       1.00      1.00      1.00        33
     walking       1.00      1.00      1.00       370

    accuracy                           1.00      1293
   macro avg       1.00      1.00      1.00      1293
weighted avg       1.00      1.00      1.00      1293

RF classifier with std:
Accuracy: 0.99923
f1_macro: 0.99582
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       1.00      0.97      0.98        33
     walking       1.00      1.00      1.00       370

    accuracy                           1.00      1293
   macro avg       1.00      0.99      1.00      1293
weighted avg       1.00      1.00      1.00   

Подивимося на параметри найкращих отриманих Random Forest класифікаторів:

In [11]:
rf_classifiers["no additional features"].best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [12]:
rf_classifiers["entropy"].best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

### Висновок:
По результату навчання моделей, що виконують класифікацію за методом опорних векторів, було отримано доволі очікуваний результат - моделі дуже добре виконують класифікацію, хоча все ж один клас з чотирьох, а саме клас stairs (розпізнавання руху користувача по сходах на основі даних акселерометра) розпізнається трохи гірше (інші класи - idle (стояння на місці), running (біг), walking (ходьба)). І найкращий результат показала модель, що навчалася на повному наборі ознак, та модель, що навчалася на мінімальному наборі ознак (мінімум, максимум, медіана, середнє арифметичне) плюс міжквартильний розмах.  
Несподіваний результат показали моделі випадкового лісу. Дві з них, а саме модель, що навчалася на мінімальному наборі ознак, та модель, що навчалася на мінімальному наборі плюс ентропія, виявилися здатними виконувати безпомилкову класифікацію. Це підтверджує те, що алгоритм випадкового лісу чудово підходить для вирішення нашої задачі.